# Initial Demo using Beer data

In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.append('..')

In [ ]:
from constants import FILE_FORMAT_MZML, FEATURE_SET_BINNED_005
from ms2lda.main import msfile_to_corpus
from ms2lda.lda_variational import VariationalLDA
from motifdb.main import get_motifset_list, post_motifsets

### Feature Extraction

In [ ]:
ms2_file = 'test_data/Beer_multibeers_1_T10_POS.mzML'
ms2_format = FILE_FORMAT_MZML

min_ms1_intensity = 0
min_ms2_intensity = 5000
mz_tol = 5.0
rt_tol = 10.0
feature_set_name = FEATURE_SET_BINNED_005

K = 300
corpus_json = os.path.join('test_data', 'beer1pos.json')

In [ ]:
lda_dict = msfile_to_corpus(ms2_file, ms2_format, min_ms1_intensity, min_ms2_intensity, mz_tol, rt_tol, feature_set_name, K, corpus_json=corpus_json)

In [ ]:
lda_dict.keys()

### Run LDA

Runs standard Variational LDA with no added motifs

In [ ]:
n_its = 100
corpus = lda_dict['corpus']
features = lda_dict['features']
doc_metadata = lda_dict['doc_metadata']

In [ ]:
vlda = VariationalLDA(corpus=corpus, K=K, normalise=1000.0)
vlda.run_vb(n_its=n_its, initialise=True)
vd = vlda.make_dictionary(
    features=features, metadata=doc_metadata, filename='test_data/beer.dict')

### MotifDB

In [ ]:
get_motifset_list()

In [ ]:
selected = [
    'Massbank library derived Mass2Motifs',
    'GNPS library derived Mass2Motifs'
]
output = post_motifsets(selected)

In [ ]:
output.keys()